$$ \delta_{gn} = -(J^T J)^{-1} J^T f(x) $$
$$ \delta_{sd} = -J^T f(x) $$
$$ J = \begin{pmatrix} \dfrac{\partial{f_1}}{\partial{\beta_1}} & \ldots & \dfrac{\partial{f_1}}{\partial{\beta_p}} \\ \vdots & \ddots & \vdots \\
 \dfrac{\partial{f_n}}{\partial{\beta_1}} & \ldots &\dfrac{\partial{f_n}}{\partial{\beta_p}}
 \end{pmatrix}, ~ f(x) = \begin{pmatrix} f_1 \\ \vdots \\ f_n \end{pmatrix}, ~ f_i = f(x_i, \beta)
$$

In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from dataset_generator import Generator


def dog_leg(X, Y, initial_w, func, jacobian_func, radius=0.01, s=0.1, max_iter=1000, tol=1e-5):
    w = initial_w

    for i in tqdm(range(max_iter)):
        J = jacobian_func(func, w, X)
        hessian = np.linalg.inv(J.T @ J)
        r = Y - f(X, w)
        gn = - np.linalg.inv(J.T @ J) @ J.T @ r
        sd = -J.T @ r
        t = np.linalg.norm(sd, 2) / np.linalg.norm(J @ sd, 2)

        dk = 0
        if np.linalg.norm(gn) <= radius:
            dk = gn
        elif np.linalg.norm(gn, 2) > t * np.linalg.norm(sd) and np.linalg.norm(sd, 2) > t * np.linalg.norm(sd):
            dk = radius * sd / np.linalg.norm(sd)
        elif np.linalg.norm(gn, 2) > t * np.linalg.norm(sd) and np.linalg.norm(sd, 2) <= t * np.linalg.norm(sd):
            dk = t * sd + s * (gn - t * sd)

        w = w + dk

    return w


def f(x, w):
    try:
        return_value = w[0] * x + w[1] * x ** 2
        # return_value = np.sin(x * w[0]) + np.sin((x ** 2) * w[1])
        return return_value
    except:
        print(f'ERROR CAUGHT. x: {x}\nw: {w}')
        exit()


def grad(f, x, w, delta=1e-6):
    n = len(w)
    wd = np.copy(w)
    ans = np.zeros(n)

    for i in range(n):
        wd[i] += delta
        fl = f(x, wd)
        wd[i] -= 2 * delta
        fr = f(x, wd)
        wd[i] += delta

        ans[i] = np.divide(fl - fr, 2 * delta)

    return ans


def jacobian_func(f, w, X):
    jacobian = np.zeros((len(X), len(w)))

    for i in range(len(jacobian)):
        jacobian[i] = grad(f, X[i], w)

    return jacobian


def loss(Dataset_X, Dataset_Y, w, f):
    return sum(np.array([(y - f(x, w)) for x, y in zip(Dataset_X, Dataset_Y)]) ** 2)


# Generate synthetic data.
# PARAMETERS
density = 8000
dots_count = 1000
variance = 0.1
dist = 2.5
weights = np.array([2, 2])
# ===========

gen = Generator(f)
X, Y, Dataset_X, Dataset_Y = gen.generate(dots_count, dist, density, variance, weights, absolute=False)

# Apply Gauss-Newton method.
initial_w = np.array([1, 1])
max_iter = 10000
p_est = dog_leg(Dataset_X, Dataset_Y, initial_w, f, jacobian_func, max_iter=max_iter)

# Plot the result.

# Plot style:
plt.style.use('default')
_ = plt.figure(figsize=(8, 8))
# ===========

print(f"Result: {p_est}")
print(f'Loss value: {loss(Dataset_X, Dataset_Y, p_est, f)}')

plt.axis('equal')
plt.scatter(Dataset_X, Dataset_Y, label='Data', color='gray', alpha=0.5, s=20.8, antialiased=True)
plt.plot(X, Y, label='Real', color='lime', antialiased=True, linewidth=3)
plt.plot(X, f(X, p_est), label='Fit', color='coral', antialiased=True, linewidth=1.7)
plt.legend()
plt.show()